In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
pd.set_option("display.max_columns", 100)

In [20]:
def OneHotEncoder(df: pd.DataFrame):
    vocab = pd.unique(df)
    if len(np.where(vocab!=vocab)[0]):
        np.delete(vocab, np.where(vocab!=vocab)[0][0])
    voc_len = len(vocab)
    voc_map = {word: ord for ord, word in enumerate(vocab)}
    def ohe(val):
        t = np.zeros(voc_len+1)
        if val != val:
            t[0] = 1
        else:
            t[voc_map[val]+1] = 1
        return t
    return lambda x: ohe(x)

def Normalizer(df: pd.DataFrame):
    mean = df.mean()
    sdv = df.std()
    def nml(val):
        if val != val:
            return 0
        else:
            return (val-mean)/sdv
    return lambda x: nml(x)

In [21]:
dataset = pd.read_csv("data/train.csv")

In [22]:
dataset.pop("Id")
# train_targets = dataset.pop("SalePrice")
labels = dataset.keys()

numlab = np.concatenate([dataset.select_dtypes(int).keys(), dataset.select_dtypes(float).keys()])
catlab = dataset.select_dtypes(np.object0).keys()

numlab = np.delete(numlab, np.where(numlab=="SalePrice"))
encoders = {}
sizes = {}

# Numerical Encoding
for l in numlab:
    norm = Normalizer(dataset[l])
    dataset[l] = dataset[l].apply(norm)
    encoders[l] = norm
    sizes[l] = dataset[l][0].shape

# Categorical Encoding
for l in catlab:
    norm = OneHotEncoder(dataset[l])
    dataset[l] = dataset[l].apply(norm)
    encoders[l] = norm
    sizes[l] = dataset[l][0].shape

In [23]:
dataset.head(3)
def merge(inp):
    op = np.array([], dtype=np.float32)
    for i in inp:
        mdim = len(np.shape(i))
        if(mdim):
            op = np.concatenate([op, i], dtype=np.float64)
        else:
            op = np.concatenate([op, [i]], dtype=np.float64)
    return op

targets = dataset.pop("SalePrice")
dataset["Encoded"] = dataset.apply(lambda x: merge(x), axis=1)
dataset.head(3)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Encoded
0,0.073350,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",-0.207948,-0.207071,"[0.0, 1.0, 0.0]","[1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.651256,-0.517023,1.050634,0.878367,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",0.509840,"[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.575228,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",-0.288554,-0.944267,-0.459145,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",-0.793162,1.161454,-0.120201,0.370207,1.107431,-0.240978,0.78947,1.227165,0.163723,-0.211381,"[0.0, 1.0, 0.0, 0.0, 0.0]",0.911897,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",-0.950901,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.992066,"[0.0, 1.0, 0.0, 0.0, 0.0]",0.311618,0.35088,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]",-0.751918,0.216429,-0.359202,-0.116299,-0.270116,-0.068668,"[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",-0.087658,-1.598563,0.138730,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0733498308209929, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
1,-0.872264,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",0.409724,-0.091855,"[0.0, 1.0, 0.0]","[1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",-0.071812,2.178881,0.156680,-0.429430,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",-0.572637,"[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",1.171591,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",-0.288554,-0.641008,0.466305,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.257052,-0.794891,-0.120201,-0.4

In [24]:
# Keep only required data
dataset_bkp = dataset.copy()
dataset = pd.DataFrame(dataset["Encoded"])
dataset["SalePrice"] = targets
dataset.head(3)

,Encoded,SalePrice
0,"[0.0733498308209929, 0.0, 1.0, 0.0, 0.0, 0.0, ...",208500
1,"[-0.8722638821913669, 0.0, 1.0, 0.0, 0.0, 0.0,...",181500
2,"[0.0733498308209929, 0.0, 1.0, 0.0, 0.0, 0.0, ...",223500


In [25]:
train, val = np.array_split(dataset, [int(0.6*len(dataset))])
print(f"Training Set: {len(train)}\nValidation Set: {len(val)}")

Training Set: 876
Validation Set: 584


In [42]:
def makeTrainDataset(df: pd.DataFrame) -> tf.data.Dataset:
    df = df.copy()
    target = df.pop("SalePrice")
    dl = {k: np.vstack(v) for k, v in df.items()}
    ds = tf.data.Dataset.from_tensor_slices((dl, target))
    ds.shuffle(len(df))
    ds = ds.batch(64)
    ds = ds.prefetch(64)
    return ds

train_ds = makeTrainDataset(train)
val_ds = makeTrainDataset(val)

In [27]:
# class FlattenTensors(layers.Layer):
#     def __init__(self) -> None:
#         super().__init__()
#     def call(self, input):
#         op = tf.convert_to_tensor([], np.float32)
#         for i in input:
#             # mdim = len(tf.shape(i))
#             rank = tf.shape(i).shape[0]
#             # if(mdim):
#             #     op = np.concatenate([op, i], dtype=np.float64)
#             # else:
#             #     try: op = np.concatenate([op, [i]], dtype=np.float64)
#             #     except: print(op, i)
#             # try: op = tf.concat([op, i], -1)
#             # except: print(op, i)
#             print(i)
#             if(rank == 1):
#                 op = tf.concat([op, i], -1)
#             else:
#                 # print(tf.convert_to_tensor([op, i]))
#                 op = tf.stack([op, i], 1)    
#         return op

In [100]:
input = tf.keras.Input(shape=(347,), name="Encoded")
x = layers.Dense(4096, activation="linear", name="d1")(input)
# x = layers.Dense(640, activation="linear", name="d2")(x)
# x = layers.Dropout(0.1)(x)
# x = layers.Dense(640, activation="linear", name="d3")(x)
# x = layers.Dense(32, activation="relu")(x)
output = layers.Dense(1)(x)

model = tf.keras.Model(input, output, name="Housing")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0125), loss="mae", metrics=["mae"])

In [101]:
model.summary()
# tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

Model: "Housing"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Encoded (InputLayer)        [(None, 347)]             0         
                                                                 
 d1 (Dense)                  (None, 4096)              1425408   
                                                                 
 dense_47 (Dense)            (None, 1)                 4097      
                                                                 
Total params: 1,429,505
Trainable params: 1,429,505
Non-trainable params: 0
_________________________________________________________________


In [102]:
model.fit(train_ds, validation_data=val_ds, epochs=250)

Epoch 1/250
14/14 [==============================] - 1s 28ms/step - loss: 181686.8281 - mae: 181686.8281 - val_loss: 172555.6562 - val_mae: 172555.6562
Epoch 2/250
14/14 [==============================] - 0s 19ms/step - loss: 170606.2031 - mae: 170606.2031 - val_loss: 153180.2812 - val_mae: 153180.2812
Epoch 3/250
14/14 [==============================] - 0s 19ms/step - loss: 142278.7188 - mae: 142278.7188 - val_loss: 113712.1172 - val_mae: 113712.1172
Epoch 4/250
14/14 [==============================] - 0s 21ms/step - loss: 93350.5781 - mae: 93350.5781 - val_loss: 58235.4727 - val_mae: 58235.4727
Epoch 5/250
14/14 [==============================] - 0s 19ms/step - loss: 50669.1797 - mae: 50669.1797 - val_loss: 40004.1016 - val_mae: 40004.1016
Epoch 6/250
14/14 [==============================] - 0s 19ms/step - loss: 35819.8281 - mae: 35819.8281 - val_loss: 26077.8672 - val_mae: 26077.8672
Epoch 7/250
14/14 [==============================] - 0s 19ms/step - loss: 25508.2422 - mae: 25508.24

In [103]:
loss = model.evaluate(val_ds)
model.save("housingmodel.h5")
model.save_weights("housingmodel_wts.wt")

10/10 [==============================] - 0s 5ms/step - loss: 17301.8535 - mae: 17301.8535


In [104]:
# Prepare Test Dataset
testset = pd.read_csv("data/test.csv")
ids = testset.pop("Id")
# testset["blk"] = np.ones(len(ids))

test_sizes = {}

# Normalize everything from old normalizers
for label, encoder in encoders.items():
    testset[label] = testset[label].apply(encoder)
    test_sizes[label] = testset[label][0].shape

# Create Encoded Inputs and truncate rest
testset["Encoded"] = testset.apply(lambda x: merge(x), axis=1)

# Make dataset
test_ds = tf.data.Dataset.from_tensor_slices({"Encoded": np.vstack(testset["Encoded"])})
test_ds = test_ds.batch(32)
test_ds = test_ds.prefetch(32)

In [105]:
a = pd.DataFrame(dataset_bkp.head(1).apply(lambda x: np.shape(x[0]), axis=0)).transpose()
a

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Encoded
0,(),"(6,)",(),(),"(3,)","(4,)","(5,)","(5,)","(3,)","(6,)","(4,)","(26,)","(10,)","(9,)","(6,)","(9,)",(),(),(),(),"(7,)","(9,)","(16,)","(17,)","(6,)",(),"(5,)","(6,)","(7,)","(6,)","(6,)","(6,)","(8,)",(),"(8,)",(),(),(),"(7,)","(6,)","(3,)","(7,)",(),(),(),(),(),(),(),(),(),(),"(5,)",(),"(8,)",(),"(7,)","(8,)",(),"(5,)",(),(),"(7,)","(7,)","(4,)",(),(),(),(),(),(),"(5,)","(6,)","(6,)",(),(),(),"(10,)","(7,)","(347,)"


In [106]:
b = pd.DataFrame(testset.head(1).apply(lambda x: np.shape(x[0]), axis=0)).transpose()
pd.concat((a,b))

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Encoded
0,(),"(6,)",(),(),"(3,)","(4,)","(5,)","(5,)","(3,)","(6,)","(4,)","(26,)","(10,)","(9,)","(6,)","(9,)",(),(),(),(),"(7,)","(9,)","(16,)","(17,)","(6,)",(),"(5,)","(6,)","(7,)","(6,)","(6,)","(6,)","(8,)",(),"(8,)",(),(),(),"(7,)","(6,)","(3,)","(7,)",(),(),(),(),(),(),(),(),(),(),"(5,)",(),"(8,)",(),"(7,)","(8,)",(),"(5,)",(),(),"(7,)","(7,)","(4,)",(),(),(),(),(),(),"(5,)","(6,)","(6,)",(),(),(),"(10,)","(7,)","(347,)"
0,(),"(6,)",(),(),"(3,)","(4,)","(5,)","(5,)","(3,)","(6,)","(4,)","(26,)","(10,)","(9,)","(6,)","(9,)",(),(),(),(),"(7,)","(9,)","(16,)","(17,)","(6,)",(),"(5,)","(6,)","(7,)","(6,)","(6,)","(6,)","(8,)",(),"(8,)",(),(),(),"(7,)","(6,)","(3,)","(7,)",(),(),(),(),(),(),(),(),(),(),"(5,)",(),"(8,)",(),"(7,)","(8,)",(),"(5,)",(),(),"(7,)","(7,)","(4,)",(),(),(),(),(),(),"(5,)","(6,)","(6,)",(),(),(),"(10,)","(7,)","(347,)"


In [107]:
labtest = dataset_bkp.keys()
labtrain = testset.keys()
for i in labtest:
    if i not in labtrain:
        print(i)

print(len(labtest), len(labtrain))

for k, v in sizes.items():
    if(test_sizes[k] != v):
        print(k, v, test_sizes[k])

80 80


In [109]:
# Predict on test set
pred_targets = model.predict(test_ds)
pred_targets

46/46 [==============================] - 0s 5ms/step


array([[120801.77],
       [164873.12],
       [190601.92],
       ...,
       [183518.64],
       [125555.77],
       [225581.23]], dtype=float32)

In [114]:
subm = pd.DataFrame()
subm["Id"] = ids
subm["SalePrice"] = pred_targets
subm.to_csv("subm_dnn.csv", index=False)